# Dataset: COVID, FLU, COLD Symptoms
Link to a dataset: https://www.kaggle.com/datasets/walterconway/covid-flu-cold-symptoms

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer, accuracy_score, f1_score

from sklearn import model_selection
from sklearn.model_selection import (
    StratifiedKFold,
    cross_validate,
    GridSearchCV,
)

Standard models used:

In [2]:
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

**fcalc** library files are located in **FCALC** folder

**FCALC** folder is located in the same directory as this notebook.

In [3]:
import FCALC.fcalc as fcalc

# Loading data

In [4]:
data_cfc = pd.read_csv('Datasets/covid_flu_cold.csv', sep=',')
data_cfc

,COUGH,MUSCLE_ACHES,TIREDNESS,SORE_THROAT,RUNNY_NOSE,STUFFY_NOSE,FEVER,NAUSEA,VOMITING,DIARRHEA,...,DIFFICULTY_BREATHING,LOSS_OF_TASTE,LOSS_OF_SMELL,ITCHY_NOSE,ITCHY_EYES,ITCHY_MOUTH,ITCHY_INNER_EAR,SNEEZING,PINK_EYE,TYPE
0,0,0,1,0,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,1,ALLERGY
1,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,1,1,1,ALLERGY
2,0,1,1,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,ALLERGY
3,0,0,0,1,1,0,0,0,0,0,...,0,1,1,0,0,1,0,1,1,ALLERGY
4,0,0,1,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,1,1,ALLERGY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44448,1,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,1,0,FLU
44449,1,1,0,1,1,0,1,1,0,1,...,0,1,1,0,0,0,0,1,0,FLU
44450,0,0,1,0,1,0,0,0,1,1,...,1,1,0,0,0,0,0,1,0,FLU
44451,0,0,0,1,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,FLU


In [5]:
one_hot = pd.get_dummies(data_cfc['TYPE']).astype(int)
one_hot.head()

,ALLERGY,COLD,COVID,FLU
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [6]:
data_cfc['COVID'] = one_hot['COVID']
data_cfc = data_cfc.drop(labels=['TYPE'], axis=1)
data_cfc.drop_duplicates(inplace=True)
data_cfc

,COUGH,MUSCLE_ACHES,TIREDNESS,SORE_THROAT,RUNNY_NOSE,STUFFY_NOSE,FEVER,NAUSEA,VOMITING,DIARRHEA,...,DIFFICULTY_BREATHING,LOSS_OF_TASTE,LOSS_OF_SMELL,ITCHY_NOSE,ITCHY_EYES,ITCHY_MOUTH,ITCHY_INNER_EAR,SNEEZING,PINK_EYE,COVID
0,0,0,1,0,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,1,1,1,0
2,0,1,1,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,0,0,0,1,1,0,0,0,0,0,...,0,1,1,0,0,1,0,1,1,0
4,0,0,1,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44448,1,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,1,0,0
44449,1,1,0,1,1,0,1,1,0,1,...,0,1,1,0,0,0,0,1,0,0
44450,0,0,1,0,1,0,0,0,1,1,...,1,1,0,0,0,0,0,1,0,0
44451,0,0,0,1,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
data_cfc.shape

(43490, 21)

In [8]:
data_cfc.COVID.value_counts(normalize=True)

COVID
0    0.952909
1    0.047091
Name: proportion, dtype: float64

In [9]:
df_positive = data_cfc[data_cfc.COVID==1].sample(n=100, random_state=49)
df_negative = data_cfc[data_cfc.COVID==0].sample(n=400, random_state=49)
df = pd.concat([df_positive, df_negative], axis=0)
df = df.sample(frac=1, random_state=49) #shuffling
df.head()

,COUGH,MUSCLE_ACHES,TIREDNESS,SORE_THROAT,RUNNY_NOSE,STUFFY_NOSE,FEVER,NAUSEA,VOMITING,DIARRHEA,...,DIFFICULTY_BREATHING,LOSS_OF_TASTE,LOSS_OF_SMELL,ITCHY_NOSE,ITCHY_EYES,ITCHY_MOUTH,ITCHY_INNER_EAR,SNEEZING,PINK_EYE,COVID
20389,0,0,1,0,1,0,0,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1522,1,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
27722,1,1,0,1,0,1,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
13680,0,0,0,0,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
18042,1,1,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
df.shape

(500, 21)

In [11]:
df.COVID.value_counts(normalize=True)

COVID
0    0.8
1    0.2
Name: proportion, dtype: float64

In [12]:
features = df.columns[0:-1]
target = df.columns[-1]

In [13]:
X = df[features]
y = df[target]

# Testing models

In [14]:
scoring = {'accuracy' : make_scorer(accuracy_score),
           'f1_macro' : make_scorer(f1_score, average='macro'),
           'f1_binary' : make_scorer(f1_score),
          }

kfold = StratifiedKFold(n_splits=10, random_state=49, shuffle=True)

In [15]:
def print_results(results):
    acc = np.round(np.mean(results['test_accuracy']), 4) 
    f1_m = np.round(np.mean(results['test_f1_macro']), 4)
    f1_b = np.round(np.mean(results['test_f1_binary']), 4)
    print(f'Accuracy = {acc:0.4f}, F1_macro = {f1_m:0.4f}, F1_binary = {f1_b:0.4f}')  

# Standard models

### Logistic regression

In [39]:
parameters = {'C' : np.linspace(1,5,101)}

model = LogisticRegression()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 101 candidates, totalling 1010 fits


{'C': 1.0}

In [40]:
model = LogisticRegression(C=best.best_params_['C'])
results = cross_validate(estimator=model,
                         X=X, y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9520, F1_macro = 0.9317, F1_binary = 0.8945


### K-Nearest Neighbours

In [18]:
parameters = {'n_neighbors' : range(4,64,3)}

model = KNeighborsClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )

best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 20 candidates, totalling 200 fits


{'n_neighbors': 37}

In [19]:
model = KNeighborsClassifier(n_neighbors=best.best_params_['n_neighbors'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9320, F1_macro = 0.9012, F1_binary = 0.8462


### Naive Bayes

##### MULTINOMIAL NB

In [20]:
parameters = {'alpha' : np.linspace(0.001,100.001,1001)}

model = MultinomialNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 1001 candidates, totalling 10010 fits


{'alpha': 2.301}

In [21]:
model = MultinomialNB(alpha=best.best_params_['alpha'])

results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9340, F1_macro = 0.9003, F1_binary = 0.8423


##### GAUSSIAN NB

In [36]:
parameters = {'var_smoothing': np.logspace(0,-9, num=100)}

model = GaussianNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


{'var_smoothing': 0.23101297000831597}

In [38]:
model = GaussianNB(var_smoothing=best.best_params_['var_smoothing'])

results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9540, F1_macro = 0.9349, F1_binary = 0.8997


##### COMPLEMENT NB

In [24]:
parameters = {'alpha' : np.linspace(0.001,100.001,1001)}

model =  ComplementNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 1001 candidates, totalling 10010 fits


{'alpha': 0.001}

In [25]:
model = ComplementNB(alpha=best.best_params_['alpha'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results) 

Accuracy = 0.9540, F1_macro = 0.9349, F1_binary = 0.8997


### Decision tree

In [26]:
parameters = {'min_samples_split' : range(2,21,2),
              'max_depth' : range(8,21,2),
              'criterion' : ['gini', 'entropy']
             }

model = DecisionTreeClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 140 candidates, totalling 1400 fits


{'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 8}

In [27]:
model = DecisionTreeClassifier(
    max_depth=best.best_params_['max_depth'], 
    min_samples_split=best.best_params_['min_samples_split'],
    criterion=best.best_params_['criterion']
)
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9540, F1_macro = 0.9349, F1_binary = 0.8997


### RandomForest

In [28]:
parameters = {'n_estimators' : range(40,101,10),
              'min_samples_split' : range(2,13,2),
              'max_depth' : range(4,15,2),
              'criterion' : ['gini', 'entropy']
             }
model = RandomForestClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=3, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 504 candidates, totalling 5040 fits
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.960) f1_binary: (test=0.889) f1_macro: (test=0.932) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=1.000) f1_binary: (test=1.000) f1_macro: (test=1.000) total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.900) f1_binary: (test=0.667) f1_macro: (test=0.804) total time=   0.0s
[CV 5/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 6/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accu

[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=80; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=80; accuracy: (test=0.880) f1_binary: (test=0.667) f1_macro: (test=0.797) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.980) f1_binary: (test=0.952) f1_macro: (test=0.970) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.900) f1_binary: (test=0.737) f1_macro: (test=0.838) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=1.000) f1_binary: (test=1.000) f1_macro: (test=1.000) total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942

[CV 8/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=60; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=60; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=60; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=70; accuracy: (test=0.960) f1_binary: (test=0.900) f1_macro: (test=0.938) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=70; accuracy: (test=0.920) f1_binary: (test=0.800) f1_macro: (test=0.875) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=70; accuracy: (test=1.000) f1_binary: (test=1.000) f1_macro: (test=1.000

[CV 8/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=40; accuracy: (test=0.920) f1_binary: (test=0.778) f1_macro: (test=0.864) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=40; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=40; accuracy: (test=0.920) f1_binary: (test=0.818) f1_macro: (test=0.883) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=50; accuracy: (test=0.980) f1_binary: (test=0.952) f1_macro: (test=0.970) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=50; accuracy: (test=0.840) f1_binary: (test=0.500) f1_macro: (test=0.702) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=50; accuracy: (test=1.000) f1_binary: (test=1.000) f1_macro: (test=1.000

[CV 7/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=90; accuracy: (test=0.900) f1_binary: (test=0.800) f1_macro: (test=0.867) total time=   0.1s
[CV 8/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=90; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=90; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.1s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=90; accuracy: (test=0.880) f1_binary: (test=0.700) f1_macro: (test=0.812) total time=   0.1s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=100; accuracy: (test=0.980) f1_binary: (test=0.952) f1_macro: (test=0.970) total time=   0.1s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=6, n_estimators=100; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.9

...

[CV 8/10] END criterion=entropy, max_depth=14, min_samples_split=8, n_estimators=80; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 9/10] END criterion=entropy, max_depth=14, min_samples_split=8, n_estimators=80; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 10/10] END criterion=entropy, max_depth=14, min_samples_split=8, n_estimators=80; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 1/10] END criterion=entropy, max_depth=14, min_samples_split=8, n_estimators=90; accuracy: (test=0.980) f1_binary: (test=0.952) f1_macro: (test=0.970) total time=   0.0s
[CV 2/10] END criterion=entropy, max_depth=14, min_samples_split=8, n_estimators=90; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 3/10] END criterion=entropy, max_depth=14, min_samples_split=8, n_estimators=90; accuracy: (test=1.000) f1_binary: (test=1.00

[CV 7/10] END criterion=entropy, max_depth=14, min_samples_split=10, n_estimators=60; accuracy: (test=0.900) f1_binary: (test=0.800) f1_macro: (test=0.867) total time=   0.0s
[CV 8/10] END criterion=entropy, max_depth=14, min_samples_split=10, n_estimators=60; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 9/10] END criterion=entropy, max_depth=14, min_samples_split=10, n_estimators=60; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 10/10] END criterion=entropy, max_depth=14, min_samples_split=10, n_estimators=60; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 1/10] END criterion=entropy, max_depth=14, min_samples_split=10, n_estimators=70; accuracy: (test=0.980) f1_binary: (test=0.952) f1_macro: (test=0.970) total time=   0.0s
[CV 2/10] END criterion=entropy, max_depth=14, min_samples_split=10, n_estimators=70; accuracy: (test=0.960) f1_binary: (tes

[CV 4/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=40; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 5/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=40; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 6/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=40; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 7/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=40; accuracy: (test=0.900) f1_binary: (test=0.800) f1_macro: (test=0.867) total time=   0.0s
[CV 8/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=40; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 9/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=40; accuracy: (test=0.960) f1_binary: (test

[CV 2/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=90; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 3/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=90; accuracy: (test=1.000) f1_binary: (test=1.000) f1_macro: (test=1.000) total time=   0.0s
[CV 4/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=90; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 5/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=90; accuracy: (test=0.960) f1_binary: (test=0.909) f1_macro: (test=0.942) total time=   0.0s
[CV 6/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=90; accuracy: (test=0.940) f1_binary: (test=0.870) f1_macro: (test=0.915) total time=   0.0s
[CV 7/10] END criterion=entropy, max_depth=14, min_samples_split=12, n_estimators=90; accuracy: (test=0.900) f1_binary: (test

{'criterion': 'gini',
 'max_depth': 6,
 'min_samples_split': 2,
 'n_estimators': 40}

In [29]:
model = RandomForestClassifier(
    n_estimators=best.best_params_['n_estimators'],
    min_samples_split=best.best_params_['min_samples_split'],
    max_depth=best.best_params_['max_depth'],
    criterion=best.best_params_['criterion']
)
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9540, F1_macro = 0.9349, F1_binary = 0.8997


# Lazy FCA

Due to the multilabel output of **BinarizedBinaryClassifier** (**1** for **True**, **0** for **false** and **-1** for **undefined**) we cannot utilize **f1_score** with **average='binary'**. Therefore, a function that interprets **undefined** as misclassification was implemented.

In [30]:
def compare_with_binary_f1(y_true, y_pred):
    y_tmp = np.concatenate(
        (np.array(y_true)[:,None],np.array(y_pred)[:,None]),
        axis=1
    )
    df_tmp = pd.DataFrame(y_tmp, columns=['y_true','y_pred'])
    df_tmp.y_true = df_tmp.y_true.astype(bool)
    df_tmp['y_new']= ~df_tmp.loc[df_tmp.y_pred==-1]['y_true']
    df_tmp.loc[df_tmp.y_pred!=-1, 'y_new'] = df_tmp.loc[df_tmp.y_pred!=-1,'y_pred'].astype(bool) 
    df_tmp.y_new = df_tmp.y_new.astype(bool)
    return(f1_score(df_tmp.y_true, df_tmp.y_new))

In [31]:
def results_fca(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    f1_binary = compare_with_binary_f1(y_true, y_pred)
    return (accuracy, f1, f1_binary)  

## BinarizedBinaryClassifier

**BinarizedBinaryClassifier** is not a **scikit** model, so **GridSearchCV** and **cross_validate** from **scikit** do not support it. Thus, we check desired parameters in **for**  loops.

In [32]:
parameters = {'alpha' : np.linspace(0, 1, 21),
              'method': ['standard','standard-support','ratio-support'],
             }

n = kfold.get_n_splits(X)

f1_best = 0
alpha_best = 0.
method_best = 'standard'

for method in parameters['method']:
    for alpha in parameters['alpha']:
        
        f1 = [0 for i in range(n)]
        for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
            bin_cls = fcalc.classifier.BinarizedBinaryClassifier(
                X.iloc[train_index].values, 
                y.iloc[train_index].to_numpy(),
                method=method,
                alpha=alpha
            )
            bin_cls.predict(X.iloc[test_index].values)
            f1[i] = f1_score(y.iloc[test_index], bin_cls.predictions, average='macro')
            print(f"[CV {i+1}/{n}] method={method}, alpha={alpha:0.2f}, f1_macro={f1[i]:0.4f}") 
                    
        f1_mean = np.mean(f1)
        print(f'f1_mean = {f1_mean:0.4f}\n')
        if(f1_mean > f1_best):
            f1_best = f1_mean
            alpha_best = alpha
            method_best = method
            
print(f"f1_best={f1_best:0.4f}, method={method_best}, alpha={alpha_best:0.2f}")

[CV 1/10] method=standard, alpha=0.00, f1_macro=0.5039
[CV 2/10] method=standard, alpha=0.00, f1_macro=0.6881
[CV 3/10] method=standard, alpha=0.00, f1_macro=0.4985
[CV 4/10] method=standard, alpha=0.00, f1_macro=0.4388
[CV 5/10] method=standard, alpha=0.00, f1_macro=0.3890
[CV 6/10] method=standard, alpha=0.00, f1_macro=0.5301
[CV 7/10] method=standard, alpha=0.00, f1_macro=0.5989
[CV 8/10] method=standard, alpha=0.00, f1_macro=0.6324
[CV 9/10] method=standard, alpha=0.00, f1_macro=0.5101
[CV 10/10] method=standard, alpha=0.00, f1_macro=0.5070
f1_mean = 0.5297

[CV 1/10] method=standard, alpha=0.05, f1_macro=0.5039
[CV 2/10] method=standard, alpha=0.05, f1_macro=0.6881
[CV 3/10] method=standard, alpha=0.05, f1_macro=0.4985
[CV 4/10] method=standard, alpha=0.05, f1_macro=0.4388
[CV 5/10] method=standard, alpha=0.05, f1_macro=0.3890
[CV 6/10] method=standard, alpha=0.05, f1_macro=0.5301
[CV 7/10] method=standard, alpha=0.05, f1_macro=0.5989
[CV 8/10] method=standard, alpha=0.05, f1_macr

[CV 6/10] method=standard, alpha=0.70, f1_macro=0.5301
[CV 7/10] method=standard, alpha=0.70, f1_macro=0.5989
[CV 8/10] method=standard, alpha=0.70, f1_macro=0.6324
[CV 9/10] method=standard, alpha=0.70, f1_macro=0.5101
[CV 10/10] method=standard, alpha=0.70, f1_macro=0.5070
f1_mean = 0.5297

[CV 1/10] method=standard, alpha=0.75, f1_macro=0.5039
[CV 2/10] method=standard, alpha=0.75, f1_macro=0.6881
[CV 3/10] method=standard, alpha=0.75, f1_macro=0.4985
[CV 4/10] method=standard, alpha=0.75, f1_macro=0.4388
[CV 5/10] method=standard, alpha=0.75, f1_macro=0.3890
[CV 6/10] method=standard, alpha=0.75, f1_macro=0.5301
[CV 7/10] method=standard, alpha=0.75, f1_macro=0.5989
[CV 8/10] method=standard, alpha=0.75, f1_macro=0.6324
[CV 9/10] method=standard, alpha=0.75, f1_macro=0.5101
[CV 10/10] method=standard, alpha=0.75, f1_macro=0.5070
f1_mean = 0.5297

[CV 1/10] method=standard, alpha=0.80, f1_macro=0.5039
[CV 2/10] method=standard, alpha=0.80, f1_macro=0.6881
[CV 3/10] method=standard, 

[CV 1/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 2/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 3/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 4/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 5/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 6/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 7/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 8/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 9/10] method=standard-support, alpha=0.35, f1_macro=0.0000
[CV 10/10] method=standard-support, alpha=0.35, f1_macro=0.0000
f1_mean = 0.0000

[CV 1/10] method=standard-support, alpha=0.40, f1_macro=0.0000
[CV 2/10] method=standard-support, alpha=0.40, f1_macro=0.0000
[CV 3/10] method=standard-support, alpha=0.40, f1_macro=0.0000
[CV 4/10] method=standard-support, alpha=0.40, f1_macro=0.0000
[CV 5/10] method=standard-support, alpha=0.40, f1_macro=0.0000
[CV 6/10] method=standard-support, a

[CV 9/10] method=standard-support, alpha=0.95, f1_macro=0.0000
[CV 10/10] method=standard-support, alpha=0.95, f1_macro=0.0000
f1_mean = 0.0000

[CV 1/10] method=standard-support, alpha=1.00, f1_macro=0.0903
[CV 2/10] method=standard-support, alpha=1.00, f1_macro=0.2289
[CV 3/10] method=standard-support, alpha=1.00, f1_macro=0.1017
[CV 4/10] method=standard-support, alpha=1.00, f1_macro=0.1594
[CV 5/10] method=standard-support, alpha=1.00, f1_macro=0.1987
[CV 6/10] method=standard-support, alpha=1.00, f1_macro=0.0919
[CV 7/10] method=standard-support, alpha=1.00, f1_macro=0.3056
[CV 8/10] method=standard-support, alpha=1.00, f1_macro=0.2784
[CV 9/10] method=standard-support, alpha=1.00, f1_macro=0.1017
[CV 10/10] method=standard-support, alpha=1.00, f1_macro=0.1000
f1_mean = 0.1657

[CV 1/10] method=ratio-support, alpha=0.00, f1_macro=0.1905
[CV 2/10] method=ratio-support, alpha=0.00, f1_macro=0.2162
[CV 3/10] method=ratio-support, alpha=0.00, f1_macro=0.2222
[CV 4/10] method=ratio-sup

[CV 1/10] method=ratio-support, alpha=0.60, f1_macro=0.1905
[CV 2/10] method=ratio-support, alpha=0.60, f1_macro=0.2162
[CV 3/10] method=ratio-support, alpha=0.60, f1_macro=0.2222
[CV 4/10] method=ratio-support, alpha=0.60, f1_macro=0.1905
[CV 5/10] method=ratio-support, alpha=0.60, f1_macro=0.2393
[CV 6/10] method=ratio-support, alpha=0.60, f1_macro=0.2100
[CV 7/10] method=ratio-support, alpha=0.60, f1_macro=0.2162
[CV 8/10] method=ratio-support, alpha=0.60, f1_macro=0.3151
[CV 9/10] method=ratio-support, alpha=0.60, f1_macro=0.1905
[CV 10/10] method=ratio-support, alpha=0.60, f1_macro=0.1972
f1_mean = 0.2188

[CV 1/10] method=ratio-support, alpha=0.65, f1_macro=0.1905
[CV 2/10] method=ratio-support, alpha=0.65, f1_macro=0.2162
[CV 3/10] method=ratio-support, alpha=0.65, f1_macro=0.2222
[CV 4/10] method=ratio-support, alpha=0.65, f1_macro=0.1905
[CV 5/10] method=ratio-support, alpha=0.65, f1_macro=0.2393
[CV 6/10] method=ratio-support, alpha=0.65, f1_macro=0.2100
[CV 7/10] method=rati

In [33]:
f1 = [0 for i in range(n)]
accuracy = [0 for i in range(n)]
f1_binary = [0 for i in range(n)]

for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    bin_cls = fcalc.classifier.BinarizedBinaryClassifier(
        X.iloc[train_index].values, 
        y.iloc[train_index].to_numpy(),
        method=method_best,
        alpha=alpha_best
    )
    bin_cls.predict(X.iloc[test_index].values)
    
    accuracy[i], f1[i], f1_binary[i] = results_fca(y.iloc[test_index], bin_cls.predictions)
    
    print(f"[CV {i+1}/{n}] method={method_best}, alpha={alpha_best:0.2f}", 
          f"accuracy={accuracy[i]:0.4f}, f1_macro={f1[i]:0.4f}, f1_binary={f1_binary[i]:0.4f}")

f1_mean = np.mean(f1)
accuracy_mean = np.mean(accuracy)
f1_binary_mean = np.mean(f1_binary)

print(f"\n accuracy={accuracy_mean:0.4f}, f1_macro={f1_mean:0.4f}, f1_binary={f1_binary_mean:0.4f}")

[CV 1/10] method=standard, alpha=0.00 accuracy=0.7600, f1_macro=0.5039, f1_binary=0.5714
[CV 2/10] method=standard, alpha=0.00 accuracy=0.7600, f1_macro=0.6881, f1_binary=0.5385
[CV 3/10] method=standard, alpha=0.00 accuracy=0.7400, f1_macro=0.4985, f1_binary=0.6061
[CV 4/10] method=standard, alpha=0.00 accuracy=0.6800, f1_macro=0.4388, f1_binary=0.5000
[CV 5/10] method=standard, alpha=0.00 accuracy=0.6600, f1_macro=0.3890, f1_binary=0.3704
[CV 6/10] method=standard, alpha=0.00 accuracy=0.8400, f1_macro=0.5301, f1_binary=0.6667
[CV 7/10] method=standard, alpha=0.00 accuracy=0.6400, f1_macro=0.5989, f1_binary=0.4706
[CV 8/10] method=standard, alpha=0.00 accuracy=0.6800, f1_macro=0.6324, f1_binary=0.5000
[CV 9/10] method=standard, alpha=0.00 accuracy=0.7800, f1_macro=0.5101, f1_binary=0.6452
[CV 10/10] method=standard, alpha=0.00 accuracy=0.7600, f1_macro=0.5070, f1_binary=0.6000

 accuracy=0.7300, f1_macro=0.5297, f1_binary=0.5469


## PatternBinaryClassifier

In [34]:
parameters = {'alpha' : np.linspace(0, 0.4, 11),
              'method': ['standard','standard-support','ratio-support'],
             }

n = kfold.get_n_splits(X)

f1_best_p = 0
alpha_best_p = 0.
method_best_p = 'standard'

for method in parameters['method']:
    for alpha in parameters['alpha']:
        
        f1 = [0 for i in range(n)]
        for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
            bin_cls = fcalc.classifier.PatternBinaryClassifier(
                X.iloc[train_index].values, 
                y.iloc[train_index].to_numpy(),
                method=method,
                alpha=alpha
            )
            bin_cls.predict(X.iloc[test_index].values)
            f1[i] = f1_score(y.iloc[test_index], bin_cls.predictions, average='macro')
            print(f"[CV {i+1}/{n}] method={method}, alpha={alpha:0.2f}, f1_macro={f1[i]:0.4f}") 
                    
        f1_mean = np.mean(f1)
        print(f'f1_mean = {f1_mean:0.4f}\n')
        if(f1_mean > f1_best_p):
            f1_best_p = f1_mean
            alpha_best_p = alpha
            method_best_p = method
            
print(f"f1_best={f1_best_p:0.4f}, method={method_best_p}, alpha={alpha_best_p:0.2f}")

[CV 1/10] method=standard, alpha=0.00, f1_macro=0.9699
[CV 2/10] method=standard, alpha=0.00, f1_macro=0.8750
[CV 3/10] method=standard, alpha=0.00, f1_macro=1.0000
[CV 4/10] method=standard, alpha=0.00, f1_macro=0.9153
[CV 5/10] method=standard, alpha=0.00, f1_macro=0.8904
[CV 6/10] method=standard, alpha=0.00, f1_macro=0.8834
[CV 7/10] method=standard, alpha=0.00, f1_macro=0.8224
[CV 8/10] method=standard, alpha=0.00, f1_macro=0.8224
[CV 9/10] method=standard, alpha=0.00, f1_macro=0.8904
[CV 10/10] method=standard, alpha=0.00, f1_macro=0.8355
f1_mean = 0.8905

[CV 1/10] method=standard, alpha=0.04, f1_macro=0.4198
[CV 2/10] method=standard, alpha=0.04, f1_macro=0.4198
[CV 3/10] method=standard, alpha=0.04, f1_macro=0.2558
[CV 4/10] method=standard, alpha=0.04, f1_macro=0.5484
[CV 5/10] method=standard, alpha=0.04, f1_macro=0.4391
[CV 6/10] method=standard, alpha=0.04, f1_macro=0.3590
[CV 7/10] method=standard, alpha=0.04, f1_macro=0.4198
[CV 8/10] method=standard, alpha=0.04, f1_macr

[CV 1/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 2/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 3/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 4/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 5/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 6/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 7/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 8/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 9/10] method=standard-support, alpha=0.12, f1_macro=0.1667
[CV 10/10] method=standard-support, alpha=0.12, f1_macro=0.1667
f1_mean = 0.1667

[CV 1/10] method=standard-support, alpha=0.16, f1_macro=0.1667
[CV 2/10] method=standard-support, alpha=0.16, f1_macro=0.1667
[CV 3/10] method=standard-support, alpha=0.16, f1_macro=0.1667
[CV 4/10] method=standard-support, alpha=0.16, f1_macro=0.1667
[CV 5/10] method=standard-support, alpha=0.16, f1_macro=0.1667
[CV 6/10] method=standard-support, a

[CV 10/10] method=ratio-support, alpha=0.16, f1_macro=0.3798
f1_mean = 0.2964

[CV 1/10] method=ratio-support, alpha=0.20, f1_macro=0.3156
[CV 2/10] method=ratio-support, alpha=0.20, f1_macro=0.2695
[CV 3/10] method=ratio-support, alpha=0.20, f1_macro=0.3156
[CV 4/10] method=ratio-support, alpha=0.20, f1_macro=0.3156
[CV 5/10] method=ratio-support, alpha=0.20, f1_macro=0.2695
[CV 6/10] method=ratio-support, alpha=0.20, f1_macro=0.2929
[CV 7/10] method=ratio-support, alpha=0.20, f1_macro=0.3156
[CV 8/10] method=ratio-support, alpha=0.20, f1_macro=0.2452
[CV 9/10] method=ratio-support, alpha=0.20, f1_macro=0.2929
[CV 10/10] method=ratio-support, alpha=0.20, f1_macro=0.3798
f1_mean = 0.3012

[CV 1/10] method=ratio-support, alpha=0.24, f1_macro=0.3156
[CV 2/10] method=ratio-support, alpha=0.24, f1_macro=0.2695
[CV 3/10] method=ratio-support, alpha=0.24, f1_macro=0.3156
[CV 4/10] method=ratio-support, alpha=0.24, f1_macro=0.3156
[CV 5/10] method=ratio-support, alpha=0.24, f1_macro=0.2695
[C

In [35]:
f1 = [0 for i in range(n)]
accuracy = [0 for i in range(n)]
f1_binary = [0 for i in range(n)]

for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    bin_cls = fcalc.classifier.PatternBinaryClassifier(
        X.iloc[train_index].values, 
        y.iloc[train_index].to_numpy(),
        method=method_best_p,
        alpha=alpha_best_p
    )
    bin_cls.predict(X.iloc[test_index].values)
    
    accuracy[i], f1[i], f1_binary[i] = results_fca(y.iloc[test_index], bin_cls.predictions)
    
    print(f"[CV {i+1}/{n}] method={method_best_p}, alpha={alpha_best_p:0.2f}", 
          f"accuracy={accuracy[i]:0.4f}, f1_macro={f1[i]:0.4f}, f1_binary={f1_binary[i]:0.4f}")

f1_mean = np.mean(f1)
accuracy_mean = np.mean(accuracy)
f1_binary_mean = np.mean(f1_binary)

print(f"\n accuracy={accuracy_mean:0.4f}, f1_macro={f1_mean:0.4f}, f1_binary={f1_binary_mean:0.4f}")

[CV 1/10] method=standard, alpha=0.00 accuracy=0.9800, f1_macro=0.9699, f1_binary=0.9524
[CV 2/10] method=standard, alpha=0.00 accuracy=0.9200, f1_macro=0.8750, f1_binary=0.8000
[CV 3/10] method=standard, alpha=0.00 accuracy=1.0000, f1_macro=1.0000, f1_binary=1.0000
[CV 4/10] method=standard, alpha=0.00 accuracy=0.9400, f1_macro=0.9153, f1_binary=0.8696
[CV 5/10] method=standard, alpha=0.00 accuracy=0.9200, f1_macro=0.8904, f1_binary=0.8333
[CV 6/10] method=standard, alpha=0.00 accuracy=0.9200, f1_macro=0.8834, f1_binary=0.8182
[CV 7/10] method=standard, alpha=0.00 accuracy=0.8600, f1_macro=0.8224, f1_binary=0.7407
[CV 8/10] method=standard, alpha=0.00 accuracy=0.8600, f1_macro=0.8224, f1_binary=0.7407
[CV 9/10] method=standard, alpha=0.00 accuracy=0.9200, f1_macro=0.8904, f1_binary=0.8333
[CV 10/10] method=standard, alpha=0.00 accuracy=0.8800, f1_macro=0.8355, f1_binary=0.7500

 accuracy=0.9200, f1_macro=0.8905, f1_binary=0.8338
